In [1]:
import os 
from pcse_gym.envs.common_env import PCSEEnv
from pcse.fileinput import CABOFileReader, YAMLCropDataProvider
from pcse.util import WOFOST80SiteDataProvider
from GreenHouseEnv.green_house import GreenHousePCSEEnv
from pcse_gym.utils.eval import EvalCallback, determine_and_log_optimum

import numpy as np

from tensorflow.summary import create_file_writer
from stable_baselines3 import A2C, SAC, PPO, TD3
from stable_baselines3.common.callbacks import BaseCallback
from datetime import datetime
from tensorflow.summary import scalar

c:\Users\Dmitry\Desktop\Lab5\ArticleGreenHouse\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:35: DeprecationWarning: ml_dtypes.float8_e4m3b11 is deprecated. Use ml_dtypes.float8_e4m3b11fnuz
  from tensorflow.tsl.python.lib.core import pywrap_ml_dtypes


In [ ]:
# Create and configure a PCSE-Gym environment
# Note: the following configuration has not been chosen for its realism
env = GreenHousePCSEEnv(
    model_config='Wofost80_NWLP_FD.conf',
    agro_config=os.path.join(os.getcwd(),'pcse_gym','envs','configs','agro','potato_cropcalendar.yaml'),
    crop_parameters=YAMLCropDataProvider(force_reload=True),
    site_parameters=WOFOST80SiteDataProvider(WAV=10,  # Initial amount of water in total soil profile [cm]
                                             NAVAILI=10,  # Amount of N available in the pool at initialization of the system [kg/ha]
                                             PAVAILI=50,  # Amount of P available in the pool at initialization of the system [kg/ha]
                                             KAVAILI=100,  # Amount of K available in the pool at initialization of the system [kg/ha]
                                             ),
    soil_parameters=CABOFileReader(os.path.join(os.getcwd(),'pcse_gym','envs','configs','soil','ec3.CAB')),
)

# Reset/initialize the environment to obtain an initial observation
o = env.reset()


##  Model trainings

PPO (Proximal Policy Optimization):
```python
total_timesteps (int): #Общее количество шагов времени, в течение которых будет произведено обучение.
callback (BaseCallback, optional):# Обратный вызов (callback) для записи промежуточных результатов обучения.
log_interval (int):# Интервал для записи логов.
eval_env (gym.Env, optional): #Среда для оценки производительности модели.
eval_freq (int): #Частота оценки (в терминах количества шагов).
n_eval_episodes (int): #Количество эпизодов для оценки модели.
#Множество других аргументов, связанных с настройками обучения и гиперпараметрами.
```


SAC (Soft Actor-Critic):
```python
total_timesteps (int): # Общее количество шагов времени, в течение которых будет произведено обучение.
callback (BaseCallback, optional):#  Обратный вызов (callback) для записи промежуточных результатов обучения.
log_interval (int): #Интервал для записи логов.
eval_env (gym.Env, optional): #Среда для оценки производительности модели.
eval_freq (int): #Частота оценки (в терминах количества шагов).
n_eval_episodes (int): #Количество эпизодов для оценки модели.
# Множество других аргументов, связанных с настройками обучения и гиперпараметрами.
```

TD3 (Twin Delayed Deep Deterministic Policy Gradient):
```python
total_timesteps (int): # Общее количество шагов времени, в течение которых будет произведено обучение.
callback (BaseCallback, optional): #Обратный вызов (callback) для записи промежуточных результатов обучения.
log_interval (int):# Интервал для записи логов.
eval_env (gym.Env, optional):# Среда для оценки производительности модели.
eval_freq (int): #Частота оценки (в терминах количества шагов).
n_eval_episodes (int): #Количество эпизодов для оценки модели.
```

In [ ]:
TENSORBOARD_LOGS_DIR=os.path.join(
    os.getcwd(),"tensorboard_logs"
    )
if not os.path.exists(TENSORBOARD_LOGS_DIR):
    os.makedirs(TENSORBOARD_LOGS_DIR)
ENV_DIR = os.path.join(TENSORBOARD_LOGS_DIR,env.__class__.name)
if not os.path.exists(ENV_DIR):
    os.makedirs(ENV_DIR)



In [ ]:
class CustomCallback(BaseCallback):
    def __init__(self, verbose=0, writer=None):
        super(CustomCallback, self).__init__(verbose)
        self.rewards = []
        self.writer = writer  # Передайте SummaryWriter

    def _on_step(self) -> bool:
        if self.model.ep_info_buffer:
            reward = self.model.ep_info_buffer[0].get('r', 0)  # Получаем значение 'r' из первого элемента или 0, если 'r' отсутствует
        else:
            reward = 0  # Обработка случая, когда очередь пуста

        self.rewards.append(reward)

        # Запишите вознаграждение в SummaryWriter
        with self.writer.as_default():
            scalar("Reward", reward, step=self.num_timesteps)

        # Здесь вы также можете записывать другие параметры среды

        return True

    def close(self) -> None:
        pass

In [ ]:
!pip freeze > requirements.txt

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%load_ext tensorboard

### SAC model

In [ ]:
sac_variants = {
    0: {'batch_size': 64, 'ent_coef': 'auto','learning_rate': 0.003,'learning_starts': 100,'buffer_size': 100000,},
    1: {'batch_size': 64, 'ent_coef': 'auto', 'learning_rate': 0.001, 'learning_starts': 100, 'buffer_size': 100000},
    2: {'batch_size': 64, 'ent_coef': 0.01, 'learning_rate': 0.003, 'learning_starts': 500, 'buffer_size': 50000},
    3: {'batch_size': 32, 'ent_coef': 'auto', 'learning_rate': 0.002, 'learning_starts': 200, 'buffer_size': 150000},
}

In [ ]:
date = datetime.now().strftime("%Y-%m-%d__%H-%M-%S")

name_model = f'SAC'

MODEL_DIR = os.path.join(ENV_DIR,name_model)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

for var in sac_variants.keys():
    name_model_var= f'var_{var}'

    MODEL_DIR_VAR = os.path.join(MODEL_DIR,name_model_var)
    if not os.path.exists(MODEL_DIR_VAR):
        os.makedirs(MODEL_DIR_VAR)

    sac_hyperparams = sac_variants[var]

    model_sac = SAC("MultiInputPolicy", env, verbose=1,**sac_hyperparams)
    # Создайте директорию для записи данных TensorBoard
    log_dir = os.path.join(
        MODEL_DIR_VAR,
        date
        )  # Замените это на путь к вашей директории
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    # Создайте SummaryWriter
    writer = create_file_writer(log_dir)

    model_sac.learn(
        total_timesteps=5e5, # Общее количество шагов времени, в течение которых будет произведено обучение.
        callback = CustomCallback(writer=writer),#  Обратный вызов (callback) для записи промежуточных результатов обучения.
        log_interval=1e2, #Интервал для записи логов.
        progress_bar=True
    )

### A2C model

In [ ]:
a2c_variants = {
    0: {'n_steps': 5,'ent_coef': 0.01,'learning_rate': 0.0007,'vf_coef': 0.25,'max_grad_norm': 0.5,},
    1: {'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.001, 'vf_coef': 0.2, 'max_grad_norm': 0.5},
    2: {'n_steps': 10, 'ent_coef': 0.015, 'learning_rate': 0.0005, 'vf_coef': 0.3, 'max_grad_norm': 0.6},
    3: {'n_steps': 5, 'ent_coef': 0.02, 'learning_rate': 0.0007, 'vf_coef': 0.15, 'max_grad_norm': 0.7},
}
# hyperparameter_variations = {
#     0: {
#         "learning_rate": 5e-4,
#         "n_steps": 2048,
#         "gamma": 0.98,
#         "gae_lambda": 0.95,
#         "ent_coef": 0.0,
#         "vf_coef": 0.5,
#         "max_grad_norm": 0.5,
#         "use_sde": True,
#         "sde_sample_freq": 50,
#         "normalize_advantage": True,
#         "stats_window_size": 50,
#     },
#     1: {
#         "learning_rate": 3e-4,
#         "n_steps": 1024,
#         "gamma": 0.99,
#         "gae_lambda": 0.96,
#         "ent_coef": 0.02,
#         "vf_coef": 0.4,
#         "max_grad_norm": 0.6,
#         "use_sde": False,
#         "sde_sample_freq": -1,
#         "normalize_advantage": False,
#         "stats_window_size": 200,
#     },
#     2: {
#         "learning_rate": 7e-4,
#         "n_steps": 4096,
#         "gamma": 0.97,
#         "gae_lambda": 0.97,
#         "ent_coef": 0.01,
#         "vf_coef": 0.6,
#         "max_grad_norm": 0.4,
#         "use_sde": True,
#         "sde_sample_freq": 100,
#         "normalize_advantage": True,
#         "stats_window_size": 100,
#     },
#     3: {
#         "learning_rate": 1e-3,
#         "n_steps": 2560,
#         "gamma": 0.95,
#         "gae_lambda": 0.98,
#         "ent_coef": 0.015,
#         "vf_coef": 0.45,
#         "max_grad_norm": 0.55,
#         "use_sde": False,
#         "sde_sample_freq": -1,
#         "normalize_advantage": True,
#         "stats_window_size": 150,
#     }
# }


In [ ]:
date = datetime.now().strftime("%Y-%m-%d__%H-%M-%S")

name_model = f'A2C'

MODEL_DIR = os.path.join(ENV_DIR,name_model)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

for var in a2c_variants.keys():
    name_model_var= f'var_{var}'

    MODEL_DIR_VAR = os.path.join(MODEL_DIR,name_model_var)
    if not os.path.exists(MODEL_DIR_VAR):
        os.makedirs(MODEL_DIR_VAR)

    a2c_hyperparams = a2c_variants[var]
    model_a2c = A2C("MultiInputPolicy", env, verbose=1,**a2c_hyperparams)

    # Создайте директорию для записи данных TensorBoard
    log_dir = os.path.join(
        MODEL_DIR_VAR,
        date
        )  # Замените это на путь к вашей директории
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    # Создайте SummaryWriter
    writer = create_file_writer(log_dir)

    model_a2c.learn(
        total_timesteps=1,#e6, # Общее количество шагов времени, в течение которых будет произведено обучение.
        callback = CustomCallback(writer=writer),#  Обратный вызов (callback) для записи промежуточных результатов обучения.
        log_interval=1e2, #Интервал для записи логов.
        progress_bar=True
    )

### PPO model

In [ ]:
ppo_variants = {
    0: {
    "learning_rate": 3e-4,
    "n_steps": 2048,
    "batch_size": 64,
    "n_epochs": 10,
    "gamma": 0.99,
    "gae_lambda": 0.95,
    "clip_range": 0.2,
    "ent_coef": 0.0,
    "vf_coef": 0.5,
    "max_grad_norm": 0.5,
    "sde_sample_freq": -1,
    "stats_window_size": 100,
},
    1: {
    "learning_rate": 2e-4,
    "n_steps": 4096,
    "batch_size": 128,
    "n_epochs": 5,
    "gamma": 0.95,
    "gae_lambda": 0.98,
    "clip_range": 0.1,
    "ent_coef": 0.02,
    "vf_coef": 0.6,
    "max_grad_norm": 0.6,
    "sde_sample_freq": -1,
    "stats_window_size": 50,
},
    2: {
    "learning_rate": 3e-4,
    "n_steps": 1024,
    "batch_size": 32,
    "n_epochs": 20,
    "gamma": 0.99,
    "gae_lambda": 0.9,
    "clip_range": 0.25,
    "ent_coef": 0.01,
    "vf_coef": 0.4,
    "max_grad_norm": 0.6,
    "sde_sample_freq": -1,
    "stats_window_size": 200,
}
,
    3: {
        "learning_rate": 5e-4,
        "n_steps": 3072,
        "batch_size": 64,
        "n_epochs": 15,
        "gamma": 0.98,
        "gae_lambda": 0.92,
        "clip_range": 0.15,
        "ent_coef": 0.015,
        "vf_coef": 0.45,
        "max_grad_norm": 0.7,
        "sde_sample_freq": -1,
        "stats_window_size": 150,
    }
}

In [ ]:
date = datetime.now().strftime("%Y-%m-%d__%H-%M-%S")

name_model = f'PPO'

MODEL_DIR = os.path.join(ENV_DIR,name_model)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
    
for var in ppo_variants.keys():
    name_model_var = f'var_{var}'

    MODEL_DIR_VAR = os.path.join(MODEL_DIR,name_model_var)
    if not os.path.exists(MODEL_DIR_VAR):
        os.makedirs(MODEL_DIR_VAR)

    ppo_hyperparams = ppo_variants[var]
    model_ppo = PPO("MultiInputPolicy", env, verbose=1,**ppo_hyperparams)

    # Создайте директорию для записи данных TensorBoard
    log_dir = os.path.join(
        MODEL_DIR_VAR,
        date
        )  # Замените это на путь к вашей директории
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    # Создайте SummaryWriter
    writer = create_file_writer(log_dir)

    model_ppo.learn(
        total_timesteps=1,#e6, # Общее количество шагов времени, в течение которых будет произведено обучение.
        callback = CustomCallback(writer=writer),#  Обратный вызов (callback) для записи промежуточных результатов обучения.
        log_interval=1e2, #Интервал для записи логов.
        progress_bar=True
    )

### TD3 model

In [ ]:
td3_variants = {
    0: {
    "learning_rate": 5e-4,
    "buffer_size": 500_000,
    "learning_starts": 200,
    "batch_size": 64,
    "tau": 0.01,
    "gamma": 0.98,
    "train_freq": (2, "episode"),
    "gradient_steps": 50,
    "policy_delay": 3,
    "target_policy_noise": 0.15,
    "target_noise_clip": 0.4,
    "stats_window_size": 50,
}
,
    1: {
    "learning_rate": 1e-3,
    "buffer_size": 1_000_000,
    "learning_starts": 100,
    "batch_size": 100,
    "tau": 0.005,
    "gamma": 0.99,
    "train_freq": (4, "step"),
    "gradient_steps": 100,
    "policy_delay": 1,
    "target_policy_noise": 0.1,
    "target_noise_clip": 0.3,
    "stats_window_size": 200,
}
,
    2: {
    "learning_rate": 2e-4,
    "buffer_size": 750_000,
    "learning_starts": 150,
    "batch_size": 75,
    "tau": 0.0075,
    "gamma": 0.97,
    "train_freq": (3, "episode"),
    "gradient_steps": 75,
    "policy_delay": 2,
    "target_policy_noise": 0.125,
    "target_noise_clip": 0.35,
    "stats_window_size": 150,
}
,
    3: {
    "learning_rate": 3e-3,
    "buffer_size": 1_250_000,
    "learning_starts": 50,
    "batch_size": 50,
    "tau": 0.0025,
    "gamma": 0.95,
    "train_freq": (1, "step"),
    "gradient_steps": 200,
    "policy_delay": 4,
    "target_policy_noise": 0.2,
    "target_noise_clip": 0.45,
    "stats_window_size": 250,
}
}

In [ ]:
date = datetime.now().strftime("%Y-%m-%d__%H-%M-%S")

name_model = f'TD3'

MODEL_DIR = os.path.join(ENV_DIR,name_model)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
    
for var in td3_variants.keys():
    name_model_var = f'var_{var}'

    MODEL_DIR_VAR = os.path.join(MODEL_DIR,name_model_var)
    if not os.path.exists(MODEL_DIR_VAR):
        os.makedirs(MODEL_DIR_VAR)
        
    td3_hyperparams = td3_variants[var]
    model_td3 = TD3("MultiInputPolicy", env, verbose=1,**td3_hyperparams)

    # Создайте директорию для записи данных TensorBoard
    log_dir = os.path.join(
        MODEL_DIR_VAR,
        date
        )  # Замените это на путь к вашей директории
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    # Создайте SummaryWriter
    writer = create_file_writer(log_dir)

    model_td3.learn(
        total_timesteps=1, #1e6, # Общее количество шагов времени, в течение которых будет произведено обучение.
        callback = CustomCallback(writer=writer),#  Обратный вызов (callback) для записи промежуточных результатов обучения.
        log_interval=1e2, #Интервал для записи логов.
        progress_bar=True
    )